# Imports

In [60]:
import os
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
import subprocess

# XML Parser

In [61]:
def replace_key_in_xml(xml_file, key_val, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for elem in root.iter():
        if elem.tag in key_val:
            key = elem.tag
            val = key_val[key]
            if key == "L3Cache_12M_16":
                for element in elem:
                    if element.tag == "Prefetcher":
                        element.text = val
            else:
                elem.text = val
    tree.write(output_file)

# Build and Run

In [62]:
def run(file_name, benchmark_path=""):
    err_file = f"./err/{file_name}.err"
    assignment_dir="/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5"
    resource_dir = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources"
    cmd = f"java -jar {resource_dir}/Tejas/jars/tejas.jar {assignment_dir}/config_tigerlake_modified.xml {assignment_dir}/outputs/{file_name}.stat"
    cmd += f" {benchmark_path}"
    with open(err_file, "w") as output_file:
        print(cmd)
        subprocess.run(cmd, shell=True, stdout=output_file)

# Constants

In [63]:
prefetchers = ["Simple", "Power4", "Bingo", "None"]
config_file_path = "./config_tigerlake.xml"
output_file = "./config_tigerlake_modified.xml"

# New Benchmarks

In [64]:
benchmark_path = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/phoronix_short_traces/"
benchmarks = os.listdir(benchmark_path)
bad_bench = ["intel-mlc_8", "stream_8"]
benchmarks = [x for x in benchmarks if x not in bad_bench and "_8" in x]

total = len(prefetchers) * len(benchmarks)
pb = tqdm(total=total)
for prefetcher in prefetchers:
    for benchmark in benchmarks:
        pb.update(1)
        print(benchmark, prefetcher)
        key_vals = {
            "isVISATrace": "false",
            "L3Cache_12M_16": prefetcher,
            "BenchmarkPath": benchmark_path+benchmark+f"/{benchmark.split('_')[0]}",
        }
        replace_key_in_xml(config_file_path, key_vals, output_file)
        file_name = f"{benchmark}_{prefetcher}"
        run(file_name)

  0%|          | 0/20 [00:00<?, ?it/s]

openssl_8 Simple
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/openssl_8_Simple.stat 


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

blender_8 Simple
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/blender_8_Simple.stat 


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

...

 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

numenta-nab_8 None
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/numenta-nab_8_None.stat 


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

compress-7zip_8 None
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/compress-7zip_8_None.stat 


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) Error in reading input packet file /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/phoronix_short_traces/compress-7zip_8/compress-7zip_0.gz
Time : 0
emulator command is : null
java.lang.Exception
	at misc.Error.showErrorAndExit(Error.java:39)
	at emulatorinterface.communication.filePacket.FilePacket.<init>(FilePacket.java:95)
	at main.Main.startCommunicationChannel(Main.java:202)
	at main.Main.main(Main.java:117)


# Old Benchmarks

In [65]:
benchmark_path = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/"
old_benchmarks = os.listdir(benchmark_path)
old_benchmarks = list(set([x.split("_")[0] for x in old_benchmarks]))

total = len(prefetchers) * len(old_benchmarks)
pb = tqdm(total=total)
for prefetcher in prefetchers:
    for benchmark in old_benchmarks:
        pb.update(1)
        print(benchmark, prefetcher)
        full_benchmark_path = benchmark_path+benchmark
        key_vals = {
            "isVISATrace": "true",
            "L3Cache_12M_16": prefetcher,
            "BenchmarkPath": full_benchmark_path
        }
        replace_key_in_xml(config_file_path, key_vals, output_file)
        file_name = f"{benchmark}_{prefetcher}"
        run(file_name, benchmark_path=full_benchmark_path)

  0%|          | 0/20 [00:00<?, ?it/s]

mcf Simple
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/mcf_Simple.stat /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/mcf


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

namd Simple
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/namd_Simple.stat /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/namd


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

...

 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

lbm None
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/lbm_None.stat /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/lbm


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

gcc None
java -jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/config_tigerlake_modified.xml /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/Assignment_5/outputs/gcc_None.stat /home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/gcc


 (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits)  (TAGE 463917)  (LOOP 1248)  (SC 58190)  (TOTAL 523355 bits 511 Kbits) 

# Statistics Parser

In [68]:
def stat_parser(stat_file_path):
    avg_mipc, mc = 0, 0
    avg_cipc, cc = 0, 0
    count = 0
    i1_hit, l1_hit, l2_hit, l3_hit = [0]*4
    with open(stat_file_path, "r") as f:
        for line in f.readlines():
            if line.startswith("IPC"):
                ipc = float(line.split("=")[1].strip().split("\t")[0])
                if "micro-ops"in line:
                    avg_mipc += ipc
                    mc += 1
                if "CISC" in line:
                    avg_cipc += ipc
                    cc += 1
            if line.startswith("L3 Hit-Rate"):
                l3_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("L2 Hit-Rate"):
                l2_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("L1 Hit-Rate"):
                l1_hit = float(line.split("=")[1].strip().split("\t")[0])
            if line.startswith("I1 Hit-Rate"):
                i1_hit = float(line.split("=")[1].strip().split("\t")[0])
    return {
        "Micro Ops IPC": avg_mipc/mc,
        "CISC IPC": avg_cipc/cc,
        "I1 Hit Rate": i1_hit,
        "L1 Hit Rate": l1_hit,
        "L2 Hit Rate": l2_hit,
        "L3 Hit Rate": l3_hit
    }

# Analyze

In [69]:
results = {}
for benchmark in benchmarks+old_benchmarks:
    results[benchmark] = {}
    for prefetcher in prefetchers:
        stat_file = f"./outputs/{benchmark}_{prefetcher}.stat"
        try:
            print(benchmark, prefetcher)
            parsed_res = stat_parser(stat_file)
            results[benchmark][prefetcher] = parsed_res
        except:
            print("File Not Found:", stat_file)
results

openssl_8 Simple
openssl_8 Power4
openssl_8 Bingo
openssl_8 None
blender_8 Simple
blender_8 Power4
blender_8 Bingo
blender_8 None
gimp_8 Simple
gimp_8 Power4
gimp_8 Bingo
gimp_8 None
numenta-nab_8 Simple
numenta-nab_8 Power4
numenta-nab_8 Bingo
numenta-nab_8 None
compress-7zip_8 Simple
File Not Found: ./outputs/compress-7zip_8_Simple.stat
compress-7zip_8 Power4
File Not Found: ./outputs/compress-7zip_8_Power4.stat
compress-7zip_8 Bingo
File Not Found: ./outputs/compress-7zip_8_Bingo.stat
compress-7zip_8 None
File Not Found: ./outputs/compress-7zip_8_None.stat
mcf Simple
mcf Power4
mcf Bingo
mcf None
namd Simple
namd Power4
namd Bingo
namd None
xalancbmk Simple
xalancbmk Power4
xalancbmk Bingo
xalancbmk None
lbm Simple
lbm Power4
lbm Bingo
lbm None
gcc Simple
gcc Power4
gcc Bingo
gcc None


{'openssl_8': {'Simple': {'Micro Ops IPC': 2.2498125,
   'CISC IPC': 1.1497125,
   'I1 Hit Rate': 0.6196407,
   'L1 Hit Rate': 0.97958034,
   'L2 Hit Rate': 0.94197726,
   'L3 Hit Rate': 0.4529387},
  'Power4': {'Micro Ops IPC': 2.1918499999999996,
   'CISC IPC': 1.1204749999999999,
   'I1 Hit Rate': 0.6228707,
   'L1 Hit Rate': 0.9799118,
   'L2 Hit Rate': 0.94231135,
   'L3 Hit Rate': 0.40105236},
  'Bingo': {'Micro Ops IPC': 2.199125,
   'CISC IPC': 1.1249875,
   'I1 Hit Rate': 0.618916,
   'L1 Hit Rate': 0.9797261,
   'L2 Hit Rate': 0.94210315,
   'L3 Hit Rate': 0.39230037},
  'None': {'Micro Ops IPC': 2.2162875,
   'CISC IPC': 1.1323124999999998,
   'I1 Hit Rate': 0.6210674,
   'L1 Hit Rate': 0.97989166,
   'L2 Hit Rate': 0.942359,
   'L3 Hit Rate': 0.35469055}},
 'blender_8': {'Simple': {'Micro Ops IPC': 1.0857875,
   'CISC IPC': 0.6359999999999999,
   'I1 Hit Rate': 0.58094096,
   'L1 Hit Rate': 0.9428488,
   'L2 Hit Rate': 0.961477,
   'L3 Hit Rate': 0.39184028},
  'Power4': {'